In [1]:
from utils import load_captions_txt

captions_dict = load_captions_txt("../data/Flickr8k_text/captions.txt")

# Poi estrai la lista flat delle caption per il tokenizer
all_captions = []
for caps in captions_dict.values():
    all_captions.extend(caps)

from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(oov_token="<unk>")
tokenizer.fit_on_texts(all_captions)

vocab_size = len(tokenizer.word_index) + 1

def max_caption_length(captions):
    return max(len(c.split()) for c in captions)

max_length = max_caption_length(all_captions)

print(f"Vocabulary size: {vocab_size}")
print(f"Max caption length: {max_length}")


Vocabulary size: 8497
Max caption length: 40


In [2]:
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add
from tensorflow.keras.optimizers import Adam

# Carica feature estratte (encoder output)
with open("features/image_features.pkl", "rb") as f:
    image_features = pickle.load(f)

# Input feature immagine
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

# Input sequenze testo (caption)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)

# Combina
decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

# Modello finale
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 40)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer         │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 40, 256)   │  2,175,232 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 2048)      │          0 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 40, 256)   │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 40)        │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    524,544 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 256)       │    525,312 │ dropout_1[0][0],  │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 256)       │          0 │ dense[0][0],      │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │     65,792 │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 8497)      │  2,183,729 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,474,609 (20.88 MB)

 Trainable params: 5,474,609 (20.88 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
from utils import data_generator_single_example

In [4]:
import tensorflow as tf
import functools

batch_size = 64

# 1. Definisci la signature per un SINGOLO esempio (senza la dimensione 'None' per il batch)
output_signature_single = (
    (
        tf.TensorSpec(shape=(2048,), dtype=tf.float32),          # Feature di UNA immagine
        tf.TensorSpec(shape=(max_length,), dtype=tf.int32)       # UNA sequenza di input
    ),
    tf.TensorSpec(shape=(vocab_size,), dtype=tf.float32)         # UN target one-hot
)


# 2. Usa functools.partial senza passare batch_size, perché il generatore non lo usa più
partial_generator = functools.partial(
    data_generator_single_example, # <-- Usa il nuovo generatore
    captions_dict, 
    image_features, 
    tokenizer, 
    max_length, 
    vocab_size
)

# 3. Crea il dataset da singoli elementi usando la signature corretta
dataset = tf.data.Dataset.from_generator(
    partial_generator,
    output_signature=output_signature_single
)

# 4. Aggiungi .batch() per creare i lotti in modo efficiente
#    Questo raggrupperà i singoli esempi in lotti di 64.
dataset = dataset.batch(batch_size)

# 5. (OPZIONALE MA RACCOMANDATO) Aggiungi .prefetch() per le performance
#    Questo permette alla CPU di preparare i dati mentre la GPU sta addestrando.
dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

# Il calcolo degli step rimane invariato
# Nota: poichè il nuovo generatore crea più esempi per caption, potresti voler aumentare gli steps
# per epoca, ma per ora lasciamolo così per coerenza.
steps_per_epoch = len(captions_dict) // batch_size


# Questo codice ora funzionerà correttamente
print("Verifica del primo batch dal dataset...")
for batch in dataset.take(1):
    (X1, X2), y = batch
    print("\n--- BATCH RICEVUTO CORRETTAMENTE DA TENSORFLOW ---")
    print("X1 shape:", X1.shape)  # Ora stamperà (64, 2048)
    print("X2 shape:", X2.shape)  # Ora stamperà (64, 34) [o la tua max_length]
    print("y shape:", y.shape)   # Ora stamperà (64, 7579) [o la tua vocab_size]

# Ora puoi usare 'dataset' e 'steps_per_epoch' direttamente in model.fit()
# Esempio:
# model.fit(dataset, epochs=10, steps_per_epoch=steps_per_epoch)

Verifica del primo batch dal dataset...

--- BATCH RICEVUTO CORRETTAMENTE DA TENSORFLOW ---
X1 shape: (64, 2048)
X2 shape: (64, 40)
y shape: (64, 8497)


In [ ]:
steps = len(captions_dict) // batch_size
#-----------------------------

print(f"Inizio addestramento con {steps} steps per epoca.")

model.fit(
    dataset,
    epochs=20,
    steps_per_epoch=steps, # Ora la variabile 'steps' esiste e ha un valore
    verbose=1
)

# Salva il modello dopo l'addestramento
print("Addestramento completato. Salvataggio del modello...")
model.save("model_caption.h5")

# Salva il tokenizer per un uso futuro
from tensorflow.keras.preprocessing.text import tokenizer_from_json

tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

print("✔️ Modello e tokenizer salvati correttamente.")

NameError: name 'steps' is not defined